# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

F. Walter  ->  F. Walter  |  ['F. Walter']
J. Li  ->  J. Li  |  ['J. Li']
J. Li  ->  J. Li  |  ['J. Li']
J. Li  ->  J. Li  |  ['J. Li']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
K. Lee  ->  K. Lee  |  ['K. Lee']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
F. Walter  ->  F. Walter  |  ['F. Walter']
A. Hughes  ->  A. Hughes  |  ['A. Hughes']


J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 69 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2503.02947


extracting tarball to tmp_2503.02947...

 done.


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2503.02947/m31frbs.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'bursttable' from 'tmp_2503.02947/bursttable.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

Found 56 bibliographic references in tmp_2503.03483/UGCA250.bbl.
Issues with the citations
syntax error in line 193: '=' expected
Retrieving document from  https://arxiv.org/e-print/2503.03493


extracting tarball to tmp_2503.03493...

 done.
Retrieving document from  https://arxiv.org/e-print/2503.03635


extracting tarball to tmp_2503.03635...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.03336-b31b1b.svg)](https://arxiv.org/abs/2503.03336) | **High angular resolution evidence of dust traps from deep ALMA Band 3 observations of LkCa15**  |
|| A. Sierra, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2025-03-06*|
|*Comments*| **|
|**Abstract**|            Dust traps are the most promising mechanisms to explain the observed substructures in protoplanetary discs. In this work, we present high-angular resolution ($\sim$60 mas, 9.4\,au) and high-sensitivity Atacama Large Millimetre/submillimetre Array (ALMA) observations at 3 mm of the transitional disc around LkCa15. The new data, combined with previous high-resolution observations at $\lambda=0.87,1.3$ mm, make LkCa15 an ideal laboratory for testing the dust trapping mechanism. We found that the width of the three rings decreases linearly with frequency, and the spectral indices show local minima at the locations of the rings, consistent with dust trap models. Multi-wavelength modelling confirms that the dust surface density and maximum grain size peak at 69 and 101\,au, and suggestive peak at 42\,au. The estimated total dust mass is between 13-250 M$_{\oplus}$, depending on the chosen opacity. The inner disc shows bright and unresolved emission at 3 mm, exhibiting a spectral index of $\alpha_{1.3-3 \rm mm} = 0.3 \pm 0.37$, and $\alpha_{\rm 3mm-3cm}$ ranging from $-0.1$ to $0.0$. These properties are consistent with free-free emission from an ionised jet or disc wind. Dust evolution models and radiative transfer calculations suggest that a viscosity coefficient of $\alpha = 10^{-3}$, a fragmentation velocity of 10 m\,s$^{-1}$, and DSHARP opacities provide the best match to the observed properties.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.03483-b31b1b.svg)](https://arxiv.org/abs/2503.03483) | **Uncovering Extraplanar Gas in UGCA 250 with the Ultra-deep MHONGOOSE Survey**  |
|| S. Kurapati, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2025-03-06*|
|*Comments*| *16 pages, 14 figures, Accepted for publication in MNRAS*|
|**Abstract**|            We use the neutral atomic hydrogen (HI) observations of the edge-on galaxy UGCA 250, taken as part of the MeerKAT HI Observations of Nearby Galactic Objects - Observing Southern Emitters (MHONGOOSE) survey to investigate the amount, morphology, and kinematics of extraplanar gas. The combination of high column density sensitivity and high spatial resolution of the survey over a large field of view is ideal for studying the underlying physics governing the extraplanar gas. These data reveal 9 additional detections within the field of view along with UGCA 250, with 8 of them being within $\sim$ 200 km s$^{-1}$ of the galaxy's systemic velocity. The galaxy seems to have a tail-like feature extending away from it in the southern direction up to $\sim$ 41 kpc (in projection). We also detect a cloud at anomalous velocities, but we did not find any optical counterpart. We construct a detailed tilted ring model for this edge-on galaxy to gain a deeper understanding of the vertical structure of its neutral hydrogen. The model that best matches the data features a thick disc with a scale height of $\sim$ 3$\pm$1 kpc and an HI mass of about 15$\%$ of the total HI mass. This extraplanar gas is detected for the first time in UGCA 250. Our analysis favours a mixed origin for the extraplanar gas in UGCA 250, likely arising from a combination of internal stellar feedback and external tidal interactions.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.03092-b31b1b.svg)](https://arxiv.org/abs/2503.03092) | **New insight into the hard X-ray emission influenced by the type-\uppercase\expandafter{\romannumeral1} bursts observed by Insight-HXMT during outburst of 4U 1636--536**  |
|| J. Q. Peng, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-03-06*|
|*Comments*| *11 pages, 8 figures, 2 tables, Published in A&A*|
|**Abstract**|            By analyzing the data from Insight-HXMT and NICER, we can determine the evolution of the significance of the hard shortage in 4U 1636--536 with its spectral state, as well as the evolution of the fraction of deficit with energy. Additionally, we investigate the possible geometry and evolution of the corona in 4U 1636-536 by combining our findings with the results of spectral analysis. We find that during the soft state, the significance of possible hard X-ray shortage in bursts is almost zero. However, in the hard state, some bursts exhibit significant shortages (>3 $\sigma$), while others do not. We attempt to establish a correlation between the significance of the hard X-ray shortage and the spectral parameters, but the data quality and the limited number of bursts prevent us from finding a strong correlation. For bursts with insignificant shortages in the soft state, their fraction of the deficit remains small. However, in the hard state, the fraction of deficit for all bursts increases with energy, regardless of the significance of the shortage of individual bursts. For bursts during the hard state, we investigate the evolution of the fraction of deficit during the bursts by stacking the peaks and decays of the bursts, respectively, and find that as the flux of the bursts decreases, the energy corresponding to the maximum of the fraction of deficit becomes progressively higher. We explore the possible geometry and evolution of the corona clued by the evolution of the fraction of deficit, which is obtained from the spectral and temporal analysis.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.03095-b31b1b.svg)](https://arxiv.org/abs/2503.03095) | **The peculiar disk evolution of 4U 1630--472 observed by Insight-HXMT during the 2022 and 2023 outbursts**  |
|| J.-Q. Peng, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-03-06*|
|*Comments*| *14 pages, 14 pages,3 tables. Published in ApJ*|
|**Abstract**|            We study the spectral properties of the black hole X-ray transient binary 4U 1630--472 during the 2022 and 2023 outbursts with Insight-HXMT observations. We find that the outbursts are in peculiar soft states. The effect of the hardening factor on the disk temperature is taken into account by kerrbb, and the flux and temperature of the disk are found to follow $F \propto T_{\rm eff}^{3.92\pm 0.13}$ and $F \propto T_{\rm eff}^{4.91\pm 1.00}$, for the two outbursts respectively. The flux-temperature relation is roughly consistent with holding a standard disk, By fitting with the p-free model, the p-value is found to have anti-correlation with disk temperature. Combined a joint diagnostic in a diagram of the relation between the non-thermal fraction and luminosity, by enclosing as well the previous outbursts, reveals a possible pattern for the disk evolution toward a slim one, and such an evolution may depend on the fraction of the non-thermal emission in the high soft state.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.03452-b31b1b.svg)](https://arxiv.org/abs/2503.03452) | **A Linear Decomposition Method to Analyze and Study Pulsar Mode Changes**  |
|| L. Hao, et al. -- incl., <mark>K. Lee</mark> |
|*Appeared on*| *2025-03-06*|
|*Comments*| *10 pages, 10 figures, accepted for publication in ApJ*|
|**Abstract**|            In this paper, we present the linear decomposition method (LDM), which we developed to detect and analyze pulsar profile variations and mode changing behaviour. We developed LDM utilizing the likelihood function approach assuming the Gaussian noise. The LDM projects pulse profiles onto significance-ordered orthonormal vector bases. We show that the method is similar to the principal component analysis (PCA), but LDM can handle more general situations. We use simulated dataset and data from the Kunming 40-m radio telescope to demonstrate the application of the LDM. We found that the LDM successfully identified mode changes for well-known mode-changing PSR B0329+54 and found a continuous pulse profile evolution for PSR B0355+54 . We also show that the LDM can be used to improve the timing precision for mode changing PSR B0329+54.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.03493-b31b1b.svg)](https://arxiv.org/abs/2503.03493) | **The Dust Polarisation and Magnetic Field Structure in the Centre of NGC253 with ALMA**  |
|| D. Belfiori, et al. -- incl., <mark>A. Hughes</mark> |
|*Appeared on*| *2025-03-06*|
|*Comments*| *22 pages, 13 Figures, 6 Tables. Accepted by A&A*|
|**Abstract**|            Magnetic fields have an impact on galaxy evolution at multiple scales. They are particularly important for starburst galaxies, where they play a crucial role in shaping the interstellar medium (ISM), influencing star formation processes and interacting with galactic outflows. The primary aim of this study is to obtain a parsec scale map of dust polarisation and B-field structure within the central starburst region of NGC253. This includes examining the relationship between the morphology of B-fields, galactic outflows and the spatial distribution of super star clusters (SSC), to understand their combined effects on the galaxy's star formation and ISM. We used ALMA full polarisation data in Bands 4 (145 GHz) and 7 (345 GHz) with resolution of 25 and 5 pc scale, respectively. According to our SED fitting analysis, the observed Band 4 emission is a combination of dust, synchrotron and free-free, while Band 7 traces only dust. The polarisation fraction (PF) of the synchrotron component is 2%, while that of the dust component is 0.3%. The B-fields orientation maps in both bands at common resolution show that the same B-fields structure is traced by dust and synchrotron emission at scales of 25 pc. The B-field morphology suggests a coupling with the multiphase outflow, while the distribution of PF in Band 7 showed to be correlated with the presence of SSC. We observed a significant anti-correlation between polarisation fraction and column density in both Bands 4 and 7. A negative correlation between PF and dispersion angle function was observed in Band 4 but was nearly absent in Band 7 at native resolution, suggesting that the tangling of B-field geometry along the plane of the sky is the main cause of depolarisation at 25 pc scales, while other factors play a role at 5 pc scales.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.03635-b31b1b.svg)](https://arxiv.org/abs/2503.03635) | **New routes for PN destruction and formation in the ISM via neutral-neutral gas-phase reactions and an extended database for reactions involving phosphorus**  |
|| M. X. Silva, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-03-06*|
|*Comments*| *12 pages, 7 figures. Accepted for publication in A&A*|
|**Abstract**|            Phosphorus plays an essential role in the chemistry of living organisms, being present in several fundamental biomolecules. The investigation of chemical reactions taking place in different astronomical environments involving phosphorus-containing molecules is essential for understanding how these species are produced and destroyed. Phosphorus monoxide (PO) and phosphorus nitride (PN) are key reservoirs of phosphorus in the Interstellar Medium (ISM). This work presents a computational study of the CPN system to identify viable reaction pathways involving atom-diatom collisions and to explore a potential destruction route for PN in the ISM. We explore the potential energy landscape of the C($\mathrm{^3P}$) + PN($^1\Sigma^+$), N($\mathrm{^4S}$) + CP($^2\Sigma^+$) and P($\mathrm{^4S}$) + CN($^2\Sigma^+$) reactions by performing high-accuracy ab initio calculations and provide their rate coefficients over a wide range of temperatures. The temperature-dependent rate coefficients were fitted to the modified Arrhenius equation: $k(T)=\alpha(T/300)^{\beta}\mathrm{exp}(-\gamma/T)$. An updated chemical network for P-bearing species was used to model the time-dependent abundances and reaction contributions of P, PO, PN, and PH during the chemical evolution of diffuse/translucent and dense clouds. The only neutral-neutral reaction capable of destroying PN without an activation energy seems to be the PN+C one. We have also shown that reactions between CP and N can yield CN and PN barrierless. Chemical models indicate that PO is a crucial species driving the gas-phase formation of PN. Typically, PO/PN ratios exceed 1, though their chemistry is influenced by photon- and cosmic-ray-induced processes. Over time in simulated dense clouds, neutral-neutral reactions such as PO + N, PH + N, P + OH, and PH + O play a significant role in determining the relative abundances of PO and PN.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.02947-b31b1b.svg)](https://arxiv.org/abs/2503.02947) | **Evidence for a hot galactic halo around the Andromeda Galaxy using fast radio bursts**  |
|| R. Anna-Thomas, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2025-03-06*|
|*Comments*| *Submitted to ApJ*|
|**Abstract**|            Fast Radio Bursts (FRBs) are millisecond-duration radio transients that serve as unique probes of extragalactic matter. We report on the discovery and localization of two FRBs piercing the Andromeda Galaxy (M31) by the realfast fast transient detection system at the Very Large Array. Their unique sightlines allow constraints on M31's electron density distribution. We localized FRB 20230903A to its host galaxy at a redshift $z=0.09$ and FRB 20230506C to a host galaxy at a redshift $z=0.39$. After accounting for the dispersion contribution from the Milky Way, the host galaxy and the intergalactic medium along the line of sight of the FRBs, we estimate that M31 alone will likely contribute between 21-217 $\mathrm{pc~cm^{-3}}$ along FRB 20230903A and between 43-338 $\mathrm{pc~cm^{-3}}$ along FRB 20230506C with a 90% confidence. We also modeled the M31 disk's contribution to the DM to determine the halo contribution. We find that the halo of M31 will contribute between 9-145 $\mathrm{pc~cm^{-3}}$ along FRB 20230903A and between 28-286 $\mathrm{pc~cm^{-3}}$ along FRB 20230506C with 90% confidence. The measured values of $\rm DM_{M31,halo}$ are consistent with the predictions from the modified Navarro-Frenk-White profile of M31's halo for a given impact parameter. The ions of the cool halo alone cannot account for the calculated $\rm DM_{M31,halo}$ and therefore this measurement presents indirect evidence of the hot halo of M31. We also suggest a possible intersection of the line of sight of FRB 20230506C with a hot baryon bridge between M31 and the Milky Way         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2503.03336/./1-Gallery_LkCa15.png', 'tmp_2503.03336/./1-Gallery_LkCa15.png', 'tmp_2503.03336/./Properties_Results.png']
copying  tmp_2503.03336/./1-Gallery_LkCa15.png to _build/html/
copying  tmp_2503.03336/./1-Gallery_LkCa15.png to _build/html/
copying  tmp_2503.03336/./Properties_Results.png to _build/html/
exported in  _build/html/2503.03336.md
    + _build/html/tmp_2503.03336/./1-Gallery_LkCa15.png
    + _build/html/tmp_2503.03336/./1-Gallery_LkCa15.png
    + _build/html/tmp_2503.03336/./Properties_Results.png
found figures ['tmp_2503.03483/./J1153-28_hiopt.png', 'tmp_2503.03483/./Channel_maps.png', 'tmp_2503.03483/./satellite_1.png', 'tmp_2503.03483/./satellite_2.png', 'tmp_2503.03483/./satellite_3.png', 'tmp_2503.03483/./satellite_4.png']
copying  tmp_2503.03483/./J1153-28_hiopt.png to _build/html/
copying  tmp_2503.03483/./Channel_maps.png to _build/html/
copying  tmp_2503.03483/./satellite_1.png to _build/html/
copying  tmp_2503.03483/./satellite_2.png to _b

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\paola}[1]{{\color{cyan} [PP: #1]}}$
$\newcommand{\paola}[1]{{\color{cyan} [PP: #1]}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# High angular resolution evidence of dust traps from deep ALMA Band 3 observations of LkCa15

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.03336-b31b1b.svg)](https://arxiv.org/abs/2503.03336)<mark>Appeared on: 2025-03-06</mark> - 

</div>
<div id="authors">

A. Sierra, et al. -- incl., <mark>M. Benisty</mark>

</div>
<div id="abstract">

**Abstract:** Dust traps are the most promising mechanisms to explain the observed substructures in protoplanetary discs. In this work, we present high-angular resolution ( $\sim$ 60 mas, 9.4 au) and high-sensitivity Atacama Large Millimetre/submillimetre Array (ALMA) observations at 3 mm of the transitional disc around LkCa15. The new data, combined with previous high-resolution observations at $\lambda=0.87,1.3$ mm, make LkCa15 an ideal laboratory for testing the dust trapping mechanism.We found that the width of the three rings decreases linearly with frequency, and the spectral indices show local minima at the locations of the rings, consistent with dust trap models. Multi-wavelength modelling confirms that the dust surface density and maximum grain size peak at 69 and 101 au, and suggestive peak at 42 au. The estimated total dust mass is between 13-250 M $_{\oplus}$ , depending on the chosen opacity.The inner disc shows bright and unresolved emission at 3 mm, exhibiting a spectral index of $\alpha_{1.3-3 \rm mm} = 0.3 \pm 0.37$ , and $\alpha_{\rm 3mm-3cm}$ ranging from $-0.1$ to $0.0$ . These properties are consistent with free-free emission from an ionised jet or disc wind.Dust evolution models and radiative transfer calculations suggest that a viscosity coefficient of $\alpha = 10^{-3}$ , a fragmentation velocity of 10 m s $^{-1}$ , and DSHARP opacities provide the best match to the observed properties.

</div>

<div id="div_fig1">

<img src="tmp_2503.03336/./1-Gallery_LkCa15.png" alt="Fig7" width="100%"/>

**Figure 7. -** Left: CLEAN image of the dust continuum data at ALMA Band 3. Middle: CLEAN image of the dust continuum data at ALMA Band 7 (Top) and Band 6 (Bottom). Right: Brightness temperature radial profile of the dust continuum data. The circular beam of 60 mas for all maps is shown in the bottom left corner of each panel. The vertical grey dashed lines indicate the positions of the three bright rings (B). The vertical coloured dashed lines indicate the position of a tentative faint ring. (*fig:CleanData*)

</div>
<div id="div_fig2">

<img src="tmp_2503.03336/./1-Gallery_LkCa15.png" alt="Fig15" width="100%"/>

**Figure 15. -** Left: CLEAN image of the dust continuum data at ALMA Band 3. Middle: CLEAN image of the dust continuum data at ALMA Band 7 (Top) and Band 6 (Bottom). Right: Brightness temperature radial profile of the dust continuum data. The circular beam of 60 mas for all maps is shown in the bottom left corner of each panel. The vertical grey dashed lines indicate the positions of the three bright rings (B). The vertical coloured dashed lines indicate the position of a tentative faint ring. (*fig:CleanData*)

</div>
<div id="div_fig3">

<img src="tmp_2503.03336/./Properties_Results.png" alt="Fig11" width="100%"/>

**Figure 11. -** Dust properties constraints from the multi-wavelength dust continuum observations of LkCa15 using the DSHARP opacities (Top panels) and Ricci's opacities (Bottom panels). The shaded regions show 1$\sigma$ and 2$\sigma$ uncertainties. The vertical dashed lines show the position of the ring locations at 42, 69, and 101 au. (*fig:DustProp*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.03336"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\HI}{\rm H{\sc i }}$
$\newcommand{\MB}{\ensuremath{\rm M_B}}$
$\newcommand{\mb}{\ensuremath{\rm M_{bar}}}$
$\newcommand{\mhi}{\ensuremath{\rm M_{HI}}}$
$\newcommand{\mg}{\ensuremath{\rm M_{g}}}$
$\newcommand{\ms}{\ensuremath{\rm M_{s}}}$
$\newcommand{\msun}{\ensuremath{\rm M_{\odot}}}$
$\newcommand{\fatm}{\ensuremath{\rm f_{atm}}}$
$\newcommand{\kms}{\ensuremath{\rm km   s^{-1}}}$
$\newcommand{\vhel}{\ensuremath{ V_{\rm hel}}}$
$\newcommand{\kpc}{\ensuremath{\rm kpc}}$
$\newcommand{\Jykms}{\ensuremath{ \rm Jy   km   s^{-1} }}$
$\newcommand{\mJybeam}{\ensuremath{ \rm mJy   beam^{-1} }}$
$\newcommand{\LCDM}{\ensuremath{\Lambda{\rm CDM}}}$
$\newcommand{\fat}{{\sc fat }}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Uncovering Extraplanar Gas in UGCA 250 with the Ultra-deep MHONGOOSE Survey

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.03483-b31b1b.svg)](https://arxiv.org/abs/2503.03483)<mark>Appeared on: 2025-03-06</mark> -  _16 pages, 14 figures, Accepted for publication in MNRAS_

</div>
<div id="authors">

S. Kurapati, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We use the neutral atomic hydrogen (H ${\sc i}$ ) observations of the edge-on galaxy UGCA 250, taken as part of the MeerKAT $\HI$ Observations of Nearby Galactic Objects - Observing Southern Emitters (MHONGOOSE) survey to investigate the amount, morphology, and kinematics of extraplanar gas. The combination of high column density sensitivity and high spatial resolution of the survey over a large field of view is ideal for studying the underlying physics governing the extraplanar gas. These data reveal 9 additional detections within the field of view along with UGCA 250, with 8 of them being within $\sim$ 200 km s $^{-1}$ of the galaxy’s systemic velocity. The galaxy seems to have a tail-like feature extending away from it in the southern direction  up to $\sim$ 41 kpc (in projection). We also detect a cloud at anomalous velocities, but we did not find any optical counterpart. We construct a detailed tilted ring model for this edge-on galaxy to gain a deeper understanding of the vertical structure of its neutral hydrogen. The model that best matches the data features a thick disc with a scale height of $\sim$ 3 $\pm$ 1 kpc and an $\HI$ mass of about 15 $\%$ of the total H ${\sc i}$ mass. This extraplanar gas is detected for the first time in UGCA 250. Our analysis favours a mixed origin for the extraplanar gas in UGCA 250, likely arising from a combination of internal stellar feedback and external tidal interactions.

</div>

<div id="div_fig1">

<img src="tmp_2503.03483/./J1153-28_hiopt.png" alt="Fig9" width="100%"/>

**Figure 9. -** . The $\HI$ integrated intensity contours are overlaid on the VST optical $g$-band image. (a) Moment 0 map of UGCA 250 and its 9 satellite galaxies. The contours are shown from data at various resolutions: 26.7$\arcsec$$\times$ 18.8$\arcsec$({\tt r10t00}) in purple with levels of 1.2$\times$10$^{19}$ cm$^{-2} \times 2^{n}$, n = 0,1,2,3 ..; 34.3$\arcsec$$\times$ 25.6$\arcsec$({\tt r15t00}) in cyan at 3.0$\times$10$^{18}$ cm$^{-2}$; 65.3$\arcsec$$\times$ 63.8$\arcsec$({\tt r05t60}) in magenta at 1.0$\times$10$^{18}$ cm$^{-2}$; and 93.8$\arcsec$$\times$ 91.7$\arcsec$({\tt r10t90}) in blue at 6.2$\times$10$^{17}$ cm$^{-2}$. (b) Moment-0 map of the galaxy UGCA 250 with the same $\HI$ contours, where the thick red contour corresponds to 1.0 $\times$10$^{20}$ cm$^{-2}$. Note that the $\HI$ column density values have not been corrected for the primary beam attenuation.  The ellipses in the bottom left corner represent the beams at each resolution. The zoomed-in moment 0 and moment 1 maps of individual satellite galaxies are shown in Figs \ref{fig:sat1} and \ref{fig:sat2} in the appendix. (c) The red, orange, and yellow show the $\HI$ integrated intensity from three different resolution data cubes: 14.4$\arcsec$$\times$ 9.8$\arcsec$, 26.7$\arcsec$$\times$ 18.8$\arcsec$, and 65.3$\arcsec$$\times$ 63.8$\arcsec$. These H{\sc i} maps are overlaid on a composite optical RGB image of UGCA  250 in the background. The creation of this image followed the technique outlined in \citet{English2017}. (*fig:mom0_overlay*)

</div>
<div id="div_fig2">

<img src="tmp_2503.03483/./Channel_maps.png" alt="Fig2" width="100%"/>

**Figure 2. -** Selected channel maps of UGCA 250 at the resolution of 34.3$\arcsec$$\times$ 25.6$\arcsec$({\tt r15t00}, $r =$ 1.5 and $t =$ 0). The heliocentric velocity in $\kms$ is indicated on top-left of each panel. Contours begin at 3$\sigma$ and increase by factors of 2. The presence of cloud and filament is evident in the channels at velocities ranging from 1587 to 1637 km s$^{-1}$ (*fig:channel_maps*)

</div>
<div id="div_fig3">

<img src="tmp_2503.03483/./satellite_1.png" alt="Fig7.1" width="25%"/><img src="tmp_2503.03483/./satellite_2.png" alt="Fig7.2" width="25%"/><img src="tmp_2503.03483/./satellite_3.png" alt="Fig7.3" width="25%"/><img src="tmp_2503.03483/./satellite_4.png" alt="Fig7.4" width="25%"/>

**Figure 7. -** The $\HI$ intensity map (moment 0) and velocity field (moment 1) of satellite galaxies. The H{\sc i} contours are at levels of 1.2$\times$10$^{19}$ cm$^{-2} \times 2^{n}$, n = 0,1,2,3 .. (*fig:sat1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.03483"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

351  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

7  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
